In [ ]:
from dotenv import load_dotenv

env_loaded = load_dotenv('../../.env-local')
env_loaded

In [2]:
from datadocai.models import CurrentTable

TRINO_CATALOG = 'postgres'
TRINO_SCHEMA = 'public'
TRINO_TABLE = 'house_pricing'

# construct the table you want to analyse
ct = CurrentTable(trino_catalog=TRINO_CATALOG,
                  trino_schema=TRINO_SCHEMA,
                  trino_table=TRINO_TABLE)

In [3]:
import os
from datadocai.database import DatabaseClient

# connect to trino
dc = DatabaseClient(host=os.getenv('TRINO_HOST'),
                    port=os.getenv('TRINO_PORT'),
                    user=os.getenv('TRINO_USER'),
                    password=os.getenv('TRINO_PASSWORD'), 
                    certificate="../../docker/trino/certificate.pem")

## Show the table informations

In [ ]:
## Show the table informations
cursor = dc.conn.cursor()
cursor.execute(
    f"SHOW CREATE TABLE {ct.trino_catalog}.{ct.trino_schema}.{ct.trino_table}")
data = cursor.fetchall()
description = cursor.description
cursor.close()

output = "\n\n"
# Add column name

for line in data:
    output += ", ".join([str(c) for c in line])
    output += "\n"

print(output)

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o', verbose=True)

In [6]:
from datadocai.metadata import TableMetadataManager
from datadocai.metadata.exporter.trino import MetadataTrinoExporter

metadata_exporter = MetadataTrinoExporter(current_table=ct, database_client=dc)

tmm = TableMetadataManager(current_table=ct, database_client=dc, llm=llm, metadata_exporter=metadata_exporter)

In [ ]:
# launch the process
tmm.process()